**Rent-a-Dent Question**

Install Optimization Package

In [3]:
pip install PulP

In [4]:
import pulp as p

Define Input Parameters

In [22]:
# create a list for the locations the cars ship from
Locations_From = [ 'Location1', 'Location2' ]

# create a list for the locations the cars ship to
Locations_To = [ 'Location3', 'Location4', 'Location5', 'Location6']

# Create a dictionary for the number of cars location1 and location2 have
Supply = {'Location1': 16,'Location2': 18}

# Create a dictionary for the number of cars location 3-6 need
Min_Capacity = { 'Location3': 5, 'Location4': 5, 'Location5': 5, 'Location6': 5 }
Max_Capacity = { 'Location3': 10, 'Location4': 10, 'Location5': 10, 'Location6': 10 }

# Create a nested dictionary for the cost
Cost = {'Location1':  {'Location3':54,'Location4':17,'Location5':23, 'Location6':30},
            'Location2':  {'Location3':24,'Location4':18,'Location5':19, 'Location6':31}}

Define the Decision Variables

In [23]:
Amount_variables = p.LpVariable.dicts("X", (Locations_From, Locations_To), lowBound=0, upBound=None, cat=p.LpContinuous)

Define the Problem

In [24]:
transportation_problem = p.LpProblem("Car_Transportation_Problem", p.LpMinimize)

Define the Objective Function

In [25]:
transportation_problem += p.lpSum(Amount_variables[i][j] * Cost[i][j]  for i in Locations_From for j in Locations_To)

Define the Constraints

In [26]:
# Supply constraints: Cars shipped from each supply location should not exceed available supply
for i in Locations_From:
    transportation_problem += p.lpSum(Amount_variables[i][j] for j in Locations_To) <= Supply[i]

# Demand constraints (min): Each demand location must receive at least the minimum capacity (5 cars)
for j in Locations_To:
    transportation_problem += p.lpSum(Amount_variables[i][j] for i in Locations_From) >= Min_Capacity[j]

# Demand constraints (max): Each demand location can receive at most the maximum capacity (10 cars)
for j in Locations_To:
    transportation_problem += p.lpSum(Amount_variables[i][j] for i in Locations_From) <= Max_Capacity[j]

# Total supply constraint: The total number of cars shipped must equal the total available supply (34 cars)
total_supply = sum(Supply.values())
transportation_problem += p.lpSum(Amount_variables[i][j] for i in Locations_From for j in Locations_To) == total_supply

Solve the problem

In [27]:
status = transportation_problem.solve()
print(f"Status: {p.LpStatus[status]}")


Status: Optimal


Print the objective function value

In [28]:
print("Total Transportation Cost = ", p.value(transportation_problem.objective))

Total Transportation Cost =  730.0


Print the value of the decision variables

In [31]:
for i in Locations_From:
    for j in Locations_To:
        print(f"Cars from {i} to {j} = {Amount_variables[i][j].varValue}")

Cars from Location1 to Location3 = 0.0
Cars from Location1 to Location4 = 10.0
Cars from Location1 to Location5 = 1.0
Cars from Location1 to Location6 = 5.0
Cars from Location2 to Location3 = 9.0
Cars from Location2 to Location4 = 0.0
Cars from Location2 to Location5 = 9.0
Cars from Location2 to Location6 = 0.0
